# Задание 1. Анализ данных и SQL-запросы

**Есть следующие датасеты c данными о продажах в онлайн-магазине в формате csv файла:**   

- Таблица `products` (товары), с полями:  
`product_id`   
`category_id`  
`product_name`	  
`stock`  

- Таблица `orders` (заказы) с полями:  
`order_id`  
`customer_id`  
`order_date`  
`total_amount`  
`payment_method`  

- Таблица `order_items` (позиции заказов) с полями:  
`item_id`  
`order_id`  
`product_id`  
`quantity`  
`price`  

- Таблица `customers` (покупатели) с полями:  
`customer_id`  
`customer_name`  
`email`  
`registration_date`  

- Таблица `categories` (категории товаров):   
`category_id`  
`category_name`  

 **Наша задача:**   
 
Написать SQL запросы для получения следующих данных из датасетов выше:  

1.1. Количество уникальных пользователей за последние 30 дней.  
1.2. Количество покупок и общая сумма продаж за последние 30 дней.  
1.3. Средний чек и средний LTV для каждой категории продуктов.  
1.4. Топ-5 продуктов по количеству покупок и выручке за последние 30 дней.  
1.5. Сегментировать пользователей на группы по частоте использования продукта и
выявить основные различия между этими группами.  

**План работы:**  

- создадим базу данных sqlite3 botdatabase.db;  
- в базе данных создадим таблицы: `products`, `orders`, `order_items`, `customers`, `categories`;  
- загрузим в созданные таблицы файлы: `products.csv`, `orders.csv`, `order_items.csv`, `customers.csv`, `categories.csv`;  
- напишем SQL запросы для получения данных.  

### 1. Создадим базу данных sqlite3 botdatabase.db

In [1]:
# загрузим нужные библиотеки
import sqlite3
import csv
import pandas as pd

In [2]:
conn = sqlite3.connect('botdatabase.db')

### 2. В базе данных создадим таблицы: products, orders, order_items, customers, categories

In [3]:
# Создаем подключение к базе данных
conn = sqlite3.connect('botdatabase.db')

# Создаем курсор для выполнения операций с базой данных
cursor = conn.cursor()

# Создаем таблицу "products"
cursor.execute('''CREATE TABLE products
                  (product_id INTEGER PRIMARY KEY,
                   category_id INTEGER,
                   product_name TEXT,
                   stock INTEGER)''')

# Создаем таблицу "orders"
cursor.execute('''CREATE TABLE orders
                  (order_id INTEGER PRIMARY KEY,
                   customer_id INTEGER,
                   order_date DATE,
                   total_amount REAL,
                   payment_method TEXT)''')

# Создаем таблицу "order_items"
cursor.execute('''CREATE TABLE order_items
                  (item_id INTEGER PRIMARY KEY,
                   order_id INTEGER,
                   product_id INTEGER,
                   quantity INTEGER,
                   price REAL)''')

# Создаем таблицу "customers"
cursor.execute('''CREATE TABLE customers
                  (customer_id INTEGER PRIMARY KEY,
                   customer_name TEXT,
                   email TEXT,
                   registration_date DATE)''')

# Создаем таблицу "categories"
cursor.execute('''CREATE TABLE categories
                  (category_id INTEGER PRIMARY KEY,
                   category_name TEXT)''')

# Сохраняем изменения в базе данных
conn.commit()

# Закрываем соединение с базой данных
conn.close()

Теперь у нас есть база данных "botdatabase.db" и пять таблиц в ней: "products", "orders", "order_items", "customers" и "categories". В полях "order_date" и "registration_date" указан тип данных "DATE", что позволит нам хранить даты в этих полях.  Мы можем проверить, что таблицы были созданы, выполнив следующий код:   

In [4]:
# Создаем подключение к базе данных
conn = sqlite3.connect('botdatabase.db')

# Создаем курсор для выполнения операций с базой данных
cursor = conn.cursor()

# Получаем список таблиц в базе данных
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# Закрываем соединение с базой данных
conn.close()

[('products',), ('orders',), ('order_items',), ('customers',), ('categories',)]


Всё ОК! Таблицы созданы.

### 3. Загрузим в созданные таблицы файлы: products.csv, orders.csv, order_items.csv, customers.csv, categories.csv

#### 3.1 Загрузим в созданную таблицу `products` файл: `products.csv`

In [5]:
# Подключение к базе данных
conn = sqlite3.connect('botdatabase.db')
cursor = conn.cursor()
# Открытие файла CSV и чтение данных
with open('products.csv', 'r') as prfile:
    reader = csv.reader(prfile)
    # Пропуск заголовка
    next(reader)
    # Загрузка данных в таблицу
    for row in reader:
        cursor.execute('INSERT INTO products VALUES (?, ?, ?, ?)', row)
# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()

#### 3.2 Загрузим в созданную таблицу `orders` файл: `orders.csv`

In [6]:
conn = sqlite3.connect('botdatabase.db')
with open('orders.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # пропускаем заголовок
    for row in reader:
        conn.execute('INSERT INTO orders VALUES (?, ?, ?, ?, ?)', row)

conn.commit()

#### 3.3 Загрузим в созданную таблицу `order_items` файл: `order_items.csv`

In [7]:
# Открываем файл CSV
with open('order_items.csv', 'r') as file:
    # Создаем объект csv.reader
    reader = csv.reader(file)
    # Пропускаем заголовок
    next(reader)
    # Создаем подключение к базе данных
    conn = sqlite3.connect('botdatabase.db')
    # Создаем курсор для выполнения запросов
    cursor = conn.cursor()
    # Используем цикл для вставки данных в таблицу
    for row in reader:
        cursor.execute('INSERT INTO order_items (order_id, product_id, quantity, price) VALUES (?, ?, ?, ?)', row[1:])
    # Сохраняем изменения
    conn.commit()
    # Закрываем соединение
    conn.close()

#### 3.4 Загрузим в созданную таблицу `customers` файл: `customers.csv`

In [8]:
# Открываем CSV-файл и загружаем данные в таблицу customers
with open('customers.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # пропускаем заголовок
     # Создаем подключение к базе данных
    conn = sqlite3.connect('botdatabase.db')
    # Создаем курсор для выполнения запросов
    cursor = conn.cursor()
    for row in reader:
        cursor.execute('''INSERT INTO customers
                          (customer_id, customer_name, email, registration_date)
                          VALUES (?, ?, ?, ?)''', row)

# Сохраняем изменения и закрываем соединение с базой данных
conn.commit()
conn.close()

#### 3.5 Загрузим в созданную таблицу `categories` файл: `categories.csv`

In [9]:
# Открываем CSV-файл и загружаем данные в таблицу
with open('categories.csv', 'r') as ctfile:
    reader = csv.reader(ctfile)
    next(reader)  # пропускаем заголовок
    # Создаем подключение к базе данных
    conn = sqlite3.connect('botdatabase.db')
    # Создаем курсор для выполнения запросов
    cursor = conn.cursor()
    for row in reader:
        cursor.execute('INSERT INTO categories (category_id, category_name) VALUES (?, ?)', row)

# Сохраняем изменения и закрываем соединение
conn.commit()
conn.close()

Теперь мы можем написать SQL запросы для получения нужных данных.

### 4. Напишем SQL запросы для получения данных

#### 4.1  Количество уникальных пользователей за последние 30 дней.

In [10]:
# подключаемся к базе данных SQLite botdatabase.db.
conn = sqlite3.connect('botdatabase.db')
cursor = conn.cursor()
# выполняем запрос на выборку всех данных из таблицы
cursor.execute("SELECT COUNT(DISTINCT customer_id) FROM orders WHERE order_date >= date('now', '-30 days')")

# получаем все данные из таблицы
rows = cursor.fetchall()

# выводим данные на экран
for row in rows:
    print(row)

# закрываем соединение с базой данных
conn.close()

(3,)


#### 4.2 Количество покупок и общая сумма продаж за последние 30 дней.

In [11]:
# подключаемся к базе данных SQLite botdatabase.db
conn = sqlite3.connect('botdatabase.db')
cursor = conn.cursor()
# выполняем запрос на выборку всех данных из таблицы
cursor.execute("SELECT COUNT(*) AS num_orders, SUM(total_amount) AS total_sales FROM orders WHERE order_date >= date('now', '-30 days')")

# получаем все данные из таблицы
rows = cursor.fetchall()

# выводим данные на экран
for row in rows:
    print(row)

# закрываем соединение с базой данных
conn.close()

(4, 500.0)


#### 4.3  Средний чек и средний LTV для каждой категории продуктов.

In [12]:
# подключаемся к базе данных SQLite botdatabase.db
conn = sqlite3.connect('botdatabase.db')
cursor = conn.cursor()
# выполняем запрос на выборку всех данных из таблицы
cursor.execute("SELECT c.category_name, AVG(o.total_amount) AS avg_order_value, AVG(o.total_amount / (SELECT COUNT(DISTINCT customer_id) FROM orders WHERE customer_id = o.customer_id)) AS avg_ltv FROM orders o JOIN order_items oi ON o.order_id = oi.order_id JOIN products p ON oi.product_id = p.product_id JOIN categories c ON p.category_id = c.category_id GROUP BY c.category_name")

# получаем все данные из таблицы
rows = cursor.fetchall()

# выводим данные на экран
for row in rows:
    print(row)

# закрываем соединение с базой данных
conn.close()

('Apparel', 150.0, 150.0)
('Electronics', 150.0, 150.0)
('Home Goods', 50.0, 50.0)


#### 4.4 Топ-5 продуктов по количеству покупок и выручке за последние 30 дней.

In [13]:
# подключаемся к базе данных SQLite botdatabase.db
conn = sqlite3.connect('botdatabase.db')
cursor = conn.cursor()
# выполняем запрос на выборку всех данных из таблицы
cursor.execute("SELECT p.product_name, SUM(oi.quantity) AS num_purchases, SUM(oi.quantity * oi.price) AS total_revenue FROM orders o JOIN order_items oi ON o.order_id = oi.order_id JOIN products p ON oi.product_id = p.product_id WHERE o.order_date >= date('now', '-30 days') GROUP BY p.product_name ORDER BY num_purchases DESC, total_revenue DESC LIMIT 5")

# получаем все данные из таблицы
rows = cursor.fetchall()

# выводим данные на экран
for row in rows:
    print(row)

# закрываем соединение с базой данных
conn.close()

('Product C', 4, 200.0)
('Product B', 3, 150.0)
('Product A', 2, 100.0)
('Product D', 1, 50.0)
